# 3.3 Pre-entrenamiento 

En esta sección, se implementarán diversos AE con el fin de pre-entrenar redes profundas. De esta forma, se espera poder regularizar un modelo determinado, buscando evitar que este sufra de overfitting. 

**Importación de módulos necesarios**

In [1]:
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense, Input
from keras.models import Sequential, Model, load_model, save_model
from keras.utils import np_utils
from keras.optimizers import SGD
import numpy as np
import warnings

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce 820M (CNMeM is disabled, cuDNN not available)


**3.3.0 Creación de conjuntos de datos a utilizar**

Análogamente a las secciones 3.1 y 3.2, se construyen los conjuntos de entrenamiento, validación y prueba.

In [2]:
# Se cargan conjutos de entrenamiento y de prueba
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Se normalizan conjuntos de datos en base a intensidad máxima de pixel
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Se transforman las imágenes de ambos conjuntos a vectores
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Se define conjunto de validación y se reestructura conjunto de entrenamiento
x_val = x_train[:5000, :]
x_train = x_train[5000:, :]
y_val = y_train[:5000]
y_train = y_train[5000:]

# Vectores y_train, y_val e y_test son transformados en matrices categóricas
y_train_ = np_utils.to_categorical(y_train)
y_val_ = np_utils.to_categorical(y_val)
y_test_ = np_utils.to_categorical(y_test)

**3.3.1 Definición de modelo a ser pre-entrenado**

En primer lugar, se diseña la red neuronal que será pre-entrenada. Está será una red FF de dos capas ocultas, las que poseerán 1000 neuronas, cada una. La función de activación de las neuronas será sigmoide. La capa de salida estará conformada por 10 neuronas (una por cada dígito), las que serán activadas por medio de la función softmax.

Para efectos comparativos, se entrenará la red sin pre-entrenamiento y se determinará el error de clasificación sobre el conjunto de pruebas. Este entrenamiento será llevado cabo a través del método de optimización *adam*, 50 epochs, batches de tamaño 25 y categorical crossentropy como función de pérdida. **Notar que a pesar de que se dió la instrucción de utilizar SGD, esto no ha sido posible, ya que al entrenar el modelo con este método, las tasas de error obtenidas (tanto de entrenamiento como de validación) eran cercanas al 91%**.

In [19]:
# Se define red vacía
model = Sequential()
# Se agregan capas ocultas
model.add(Dense(1000, activation='sigmoid', input_shape=(784,)))
model.add(Dense(1000, activation='sigmoid'))
# Se agrega capa de salida
model.add(Dense(10, activation='softmax'))
# Se definen parámetros de entrenamiento
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Se entrena modelo
model.fit(x_train, y_train_, epochs=50, batch_size=25, shuffle=True, validation_data=(x_val, y_val_))

Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 41s - loss: 0.3219 - acc: 0.9007 - val_loss: 0.1339 - val_acc: 0.9584
Epoch 2/50
55000/55000 [==============================] - 41s - loss: 0.1259 - acc: 0.9615 - val_loss: 0.1091 - val_acc: 0.9682
Epoch 3/50
55000/55000 [==============================] - 41s - loss: 0.0790 - acc: 0.9755 - val_loss: 0.0898 - val_acc: 0.9728
Epoch 4/50
55000/55000 [==============================] - 42s - loss: 0.0542 - acc: 0.9827 - val_loss: 0.0759 - val_acc: 0.9776
Epoch 5/50
55000/55000 [==============================] - 42s - loss: 0.0388 - acc: 0.9874 - val_loss: 0.0703 - val_acc: 0.9802
Epoch 6/50
55000/55000 [==============================] - 43s - loss: 0.0297 - acc: 0.9900 - val_loss: 0.0924 - val_acc: 0.9744
Epoch 7/50
55000/55000 [==============================] - 43s - loss: 0.0229 - acc: 0.9923 - val_loss: 0.0767 - val_acc: 0.9814
Epoch 8/50
55000/55000 [==============================]

In [20]:
# Se determina error de clasificación sobre conjunto de pruebas
model_scores = model.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_scores[1]) * 100, '%'

 9760/10000 [============================>.] - ETA: 0s
Error de clasificacion sobre conjunto de pruebas:  1.79 %


Así, el entrenamiento finaliza con función de pérdida 0,0019 y error de clasificación de entrenamiento de un 0,04%. Además, el error de clasificación sobre el conjunto de pruebas es de un 1,79%.

**3.3.2 Pre-entrenando el modelo**

'Con el fin de mejorar el error de clasificación, se pre-entrenará la red diseñada en la sección anterior. Como dicha red posee dos capas ocultas, se utilizarán dos autoencoders, cada uno encargado de pre-entrenar los pesos de una y sólo una de las capas. Para llevar a cabo esta tarea, se implementa la función *pre_training*, la que recibe el nombre de la función de activación a ser utilizada tanto en los autoencoders como en el modelo. Como resultado de este proceso, el modelo de la sección anterior será entrenado de tal manera que los pesos de cada capa oculta serán los mismos que pre-entrenó el autoencoder respectivo.

Notar que el modelo en estrenado durante sólo 20 epochs, a diferencia del entrenamiento del modelo de la sección anterior, para el cual se emplearon 50 epochs. Se ha tomado esta medida con el fin de comprobar la eficacia de pre-entrenar los pesos. Tal eficacia debiese manifestarse a través de la obtención de una tasa de error de clasificación cercana similar a la conseguida sin pre-entrenar.

In [17]:
# pre_training permite el pre-entrenamiento de una red FF
def pre_training(encoder_activation, x_train, x_val, decoder_activation=None, dAE=''):
    
    if not decoder_activation:
        decoder_activation = encoder_activation
    
    # Se implementa primer autoencoder, encargado de pre-entrenar primera capa oculta
    # Se define input a ser recibido por autoencoder
    input_img1 = Input(shape=(784, ))
    # Se define capa encoder 
    encoded1 = Dense(1000, activation=encoder_activation)(input_img1)
    # Se define capa decoder
    decoded1 = Dense(784, activation=decoder_activation)(encoded1)
    # Se define autoencoder
    autoencoder1 = Model(input=input_img1, output=decoded1)
    # Se define encoder por separado
    encoder1 = Model(input_img1, output=encoded1)
    # Se definen parámetros de pre-entrenamiento
    autoencoder1.compile(optimizer='adam', loss='binary_crossentropy')
    # Se pre-entrena primer autoencoder
    autoencoder1.fit(x_train, x_train, epochs=50, batch_size=25, shuffle=True, verbose=0, validation_data=(x_val, x_val))
    encoded_input1 = Input(shape=(1000, ))
    # Se guardan autoencoder y encoder en archivos
    autoencoder1.save('autoencoder_1_' + encoder_activation + dAE + '.h5')
    encoder1.save('encoder_1_' + encoder_activation + dAE + '.h5')
    
    # Se implementa segundo autoencoder, encargado de pre-entrenar segunda capa oculta
    # Primero, se obtienen las salidas generadas por el encoder del primer autoencoder para cada conjunto de datos
    x_train_encoded1 = encoder1.predict(x_train)
    x_val_encoded1 = encoder1.predict(x_val)
    x_test_encoded1 = encoder1.predict(x_test)

    # Se define input a ser recibido por autoencoder
    input_img2 = Input(shape=(1000, ))
    # Se define capa encoder
    encoded2 = Dense(1000, activation=encoder_activation)(input_img2)
    # Se define capa decoder
    decoded2 = Dense(1000, activation=decoder_activation)(encoded2)
    # Se define autoencoder
    autoencoder2 = Model(input=input_img2, output=decoded2)
    # Se define encoder por separado
    encoder2 = Model(input=input_img2, output=encoded2)
    # Se especifican parámetros de optimización
    autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')
    # Se pre-entrena segundo autoencoder
    autoencoder2.fit(x_train_encoded1, x_train_encoded1, epochs=50, batch_size=25, shuffle=True, verbose=0,
                 validation_data=(x_val_encoded1, x_val_encoded1))
    encoded_input2 = Input(shape=(1000, ))
    autoencoder2.save('autoencoder_2_' + encoder_activation + dAE + '.h5')
    encoder2.save('encoder_2_' + encoder_activation + dAE + '.h5')

    # Finalmente, se entrena modelo a partir de pesos pre-entrenados
    model = Sequential()
    # Se agrega primera capa oculta
    model.add(Dense(1000, activation=encoder_activation, input_shape=(784, )))
    # Los pesos de la primera capa oculta son igualados a pesos pre-entrenados por primer autoencoder
    model.layers[-1].set_weights(autoencoder1.layers[1].get_weights())
    # Se agrega segunda capa oculta
    model.add(Dense(1000, activation=decoder_activation))
    # Los pesos de la segunda capa oculta son igualados a pesos pre-entrenados por segundo autoencoder
    model.layers[-1].set_weights(autoencoder2.layers[1].get_weights())
    # Se define capa de salida
    model.add(Dense(10, activation='softmax'))
    # Se especifican parámetros de optimización
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # Se entrena modeo
    model.fit(x_train, y_train_, epochs=20, batch_size=25, shuffle=True, verbose=1 , validation_data=(x_val, y_val_))
    # Se guarda modelo entrenado en archivo
    model.save('Net-784x1000x1000x10-finetunned_' + encoder_activation + dAE + '.h5')

Así, se ejecuta el proceso descrito sobre el modelo de la sección anterior utilizando la función de activación sigmoide tanto en los autoencoders como en las capas ocultas del modelo.

In [32]:
warnings.filterwarnings('ignore')

pre_training('sigmoid', x_train, x_val)
model_sigmoid = load_model('Net-784x1000x1000x10-finetunned_sigmoid.h5')
model_sigmoid_scores = model_sigmoid.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_sigmoid_scores[1]) * 100, '%'

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 41s - loss: 0.2367 - acc: 0.9277 - val_loss: 0.1243 - val_acc: 0.9630
Epoch 2/20
55000/55000 [==============================] - 41s - loss: 0.0889 - acc: 0.9722 - val_loss: 0.0866 - val_acc: 0.9728
Epoch 3/20
55000/55000 [==============================] - 41s - loss: 0.0537 - acc: 0.9831 - val_loss: 0.0817 - val_acc: 0.9742
Epoch 4/20
55000/55000 [==============================] - 41s - loss: 0.0360 - acc: 0.9883 - val_loss: 0.0769 - val_acc: 0.9796
Epoch 5/20
55000/55000 [==============================] - 41s - loss: 0.0243 - acc: 0.9920 - val_loss: 0.0764 - val_acc: 0.9802
Epoch 6/20
55000/55000 [==============================] - 41s - loss: 0.0198 - acc: 0.9935 - val_loss: 0.0696 - val_acc: 0.9816
Epoch 7/20
55000/55000 [==============================] - 41s - loss: 0.0156 - acc: 0.9944 - val_loss: 0.0691 - val_acc: 0.9826
Epoch 8/20
55000/55000 [==============================]

Se ve entonces que el entrenamiento finaliza con una función de pérdida de 0,0045 y error de clasificación de 0,15%, valores superiores a los obtenidos sin pre-entrenamiento, a pesar de que los valores iniciales tanto de pérdida como de error fueron más bajos en la primera iteración con respecto al modelo original, lo que quiere decir que, en este caso, los pesos se econtraban más cerca de sus valores óptimos al momento de iniciarse el entrenamiento. 

Se tiene también que el error de clasificación sobre el conjunto de pruebas aumenta a un 2,29%, por lo que los resultados obtenidos no son del todo satisfactorios. Dada esta situación, es interesante estudiar los resultados que se obtienen al utilizar otras funciones de activación (ver sección 3.3.4).

**3.3.3 Pre-entrenamiento vía *denoising autoencoder***

Con el fin de mejorar el poder de generalización del clasificador, se introduce ruido en las imágenes, repitiendo el proceso ejecutado en la sección anterior, pero pre-entrenando los pesos a partir de las imágenes corruptas. Así, se "corrompen" los conjuntos de entrenamiento y de validación tal y como se hizo en el problema 3.2., con el parámetro devst seteado en 0.1. 

In [18]:
# Se genera versión corrupta de imágenes de cada conjunto de datos

# Se utiliza desviación estándar 0.1
devst = 0.1

# Se genera versión corrupta de datos de entrenamiento
noise_mask_train = devst * np.random.standard_normal(size=x_train.shape)
noisy_x_train = x_train + noise_mask_train

# Se genera versión corrupta de datos de validación
noise_mask_val = devst * np.random.standard_normal(size=x_val.shape)
noisy_x_val = x_val + noise_mask_val

Generados los nuevos conjuntos de datos, se procede a pre-entrenar y posteriormente entrenar el modelo original por medio de dichos conjuntos.

In [19]:
warnings.filterwarnings('ignore')

pre_training('sigmoid', noisy_x_train, noisy_x_val, dAE='_dAE')
model_sigmoid_dAE = load_model('Net-784x1000x1000x10-finetunned_sigmoid_dAE.h5')
model_sigmoid_dAE_scores = model_sigmoid_dAE.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_sigmoid_dAE_scores[1]) * 100, '%'

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 41s - loss: 0.3665 - acc: 0.8882 - val_loss: 0.2077 - val_acc: 0.9348
Epoch 2/20
55000/55000 [==============================] - 41s - loss: 0.1469 - acc: 0.9541 - val_loss: 0.1395 - val_acc: 0.9568
Epoch 3/20
55000/55000 [==============================] - 41s - loss: 0.0744 - acc: 0.9769 - val_loss: 0.1282 - val_acc: 0.9642
Epoch 4/20
55000/55000 [==============================] - 41s - loss: 0.0376 - acc: 0.9881 - val_loss: 0.1397 - val_acc: 0.9618
Epoch 5/20
55000/55000 [==============================] - 43s - loss: 0.0224 - acc: 0.9926 - val_loss: 0.1295 - val_acc: 0.9678
Epoch 6/20
55000/55000 [==============================] - 38s - loss: 0.0161 - acc: 0.9947 - val_loss: 0.1412 - val_acc: 0.9704
Epoch 7/20
55000/55000 [==============================] - 38s - loss: 0.0142 - acc: 0.9954 - val_loss: 0.1220 - val_acc: 0.9750
Epoch 8/20
55000/55000 [==============================]

Los resultados muestran que el entrenamiento termina con función de pérdida 0,0038 y error de clasificación de 0,11%, mejorando levemente los resultados de la sección anterior, a pesar de que los valores iniciales de la pérdida y el error fueron más altos en este último caso. A pesar de esto, el error de clasificación sobre el conjunto de pruebas es de un 2,4%, siendo superior al error obtenido en la sección 3.3.2 por un escaso margen. 

De esta forma, el haber agregado ruido a los datos no ha ofrecido mejoras. Quizás hubiese resultado interesante trabajar con conjuntos de datos formados tanto por imágenes corruptas como imágenes normales.  

**3.3.4 Pre-entrenamiento con tanh y ReLU**

Es de interés conocer como varían los resultados de la sección 3.3.2 al modificar la función de activación tanto en el encoder como el decoder del autoencoder construído. Primero, se estudia el efector de utilizar la función tanh.

In [34]:
warnings.filterwarnings('ignore')

pre_training('tanh', x_train, x_val)
model_tanh = load_model('Net-784x1000x1000x10-finetunned_tanh.h5')
model_tanh_scores = model_tanh.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_tanh_scores[1]) * 100, '%'

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 38s - loss: 0.2519 - acc: 0.9237 - val_loss: 0.1382 - val_acc: 0.9596
Epoch 2/20
55000/55000 [==============================] - 38s - loss: 0.1325 - acc: 0.9594 - val_loss: 0.1143 - val_acc: 0.9668
Epoch 3/20
55000/55000 [==============================] - 38s - loss: 0.1108 - acc: 0.9659 - val_loss: 0.1159 - val_acc: 0.9686
Epoch 4/20
55000/55000 [==============================] - 39s - loss: 0.0943 - acc: 0.9705 - val_loss: 0.1024 - val_acc: 0.9726
Epoch 5/20
55000/55000 [==============================] - 39s - loss: 0.0826 - acc: 0.9747 - val_loss: 0.1101 - val_acc: 0.9654
Epoch 6/20
55000/55000 [==============================] - 39s - loss: 0.0747 - acc: 0.9769 - val_loss: 0.1044 - val_acc: 0.9716
Epoch 7/20
55000/55000 [==============================] - 40s - loss: 0.0712 - acc: 0.9767 - val_loss: 0.1284 - val_acc: 0.9684
Epoch 8/20
55000/55000 [==============================]

Se observa que la utilización de tanh tanto en el encoder como el decoder empeora los resultados en comparación con sigmoide: La función de pérdida y el error de clasificación poseen valores iniciales y finales más altos con tanh que con sigmoide. También, el error de clasificación sobre el conjunto de pruebas aumenta de 1,79% con sigmoide a 2,82% con tanh, por lo que queda en evidencia que no es conveniente trabajar con esta última función para resolver el problema planteado. 

De forma análoga, se estudia el efecto de utilizar la función ReLU como función de activación. Recordando que previamente se obtuvieron  malos resultados al utilizar ReLU en el decoder (problema 3.1), se usará esta función sólo en el encoder, mientras que en el decoder nuevamente se usará sigmoide.

In [20]:
warnings.filterwarnings('ignore')

pre_training('relu', x_train, x_val, decoder_activation='sigmoid')
model_relu = load_model('Net-784x1000x1000x10-finetunned_relu.h5')
model_relu_scores = model_relu.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_relu_scores[1]) * 100, '%'

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 38s - loss: 0.1843 - acc: 0.9433 - val_loss: 0.1020 - val_acc: 0.9676
Epoch 2/20
55000/55000 [==============================] - 38s - loss: 0.0687 - acc: 0.9785 - val_loss: 0.0774 - val_acc: 0.9780
Epoch 3/20
55000/55000 [==============================] - 38s - loss: 0.0447 - acc: 0.9852 - val_loss: 0.0861 - val_acc: 0.9780
Epoch 4/20
55000/55000 [==============================] - 40s - loss: 0.0313 - acc: 0.9898 - val_loss: 0.0746 - val_acc: 0.9788
Epoch 5/20
55000/55000 [==============================] - 40s - loss: 0.0247 - acc: 0.9920 - val_loss: 0.0782 - val_acc: 0.9804
Epoch 6/20
55000/55000 [==============================] - 40s - loss: 0.0190 - acc: 0.9939 - val_loss: 0.0871 - val_acc: 0.9794
Epoch 7/20
55000/55000 [==============================] - 40s - loss: 0.0157 - acc: 0.9949 - val_loss: 0.0916 - val_acc: 0.9804
Epoch 8/20
55000/55000 [==============================]

Los resultados muestran que, inicialmente, la función de pérdida y el error de clasificación en el entrenamiento poseen valores más bajos respecto al caso en que la función de activación es sigmoide. Lamentablemente, los valores finales de estas mismas métricas terminan siendo mayores en este caso. No obstante, el error de clasificación sobre el conjunto de pruebas resulta ser del 1,82%, bastante cercano al 1,79% que se obtiene con sigmoide.

De esta manera, el pre-entrenamiento del modelo original a partir de un autoencoder que utiliza la función ReLU en el encoder y sigmoide en el decoder logra obtener los mejores resultados en términos del error de clasificación sobre el conjunto de pruebas respecto a todos los pre-entrenamientos estudiados.